In [11]:
import subprocess
import time
import re
import datetime

In [12]:
IPs=[['192.168.0.1', 'Wifi'],
     ['192.168.1.1', 'ONU'],
     ['192.168.0.56', 'Repeater'],
     ['172.16.109.193', 'Gateway'],
     ['172.16.0.1', 'Old DNS'],
     ['202.171.240.26', 'New DNS'],
     ['8.8.8.8', 'Google DNS'],
     ['1.1.1.1', 'Cloudflare DNS'],
     ['40.126.17.132', 'Microsoft Chennai'],
     ['142.250.196.46', 'Google Chennai'],
     ['155.133.224.2', 'Valve Maharastra'],
     ['202.78.239.186', 'Alliance Kolkata'],
     ['57.144.141.32', 'Facebook Kolkata'],
     ['72.25.64.32', 'NVIDIA California'],
     ['140.82.112.26', 'GitHub California'],
     ['20.198.167.116', 'MS-AZC Singapore'],
     ['45.129.229.1','iCDN Indonesia']]
count=10
timeout=1000
packet_size=128

In [13]:
processes=[]
for ip in IPs:
    processes.append(subprocess.Popen(["ping",ip[0],"-n",str(count),"-w",str(timeout),"-l",str(packet_size)],
                                      stdout=subprocess.PIPE,text=True,shell=True))
time.sleep(count*(timeout/1000)*(1+len(IPs)*0.05))
outputs=[]
for process in processes:
    outputs.append(process.stdout)

In [14]:
summary=[datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")]
for output in outputs:
    ip=IPs[outputs.index(output)]
    temp=f'Pinged {ip[0].ljust(15)} [{ip[1].center(20)}] w/ [P{packet_size}] Avg='
    loss="0%"
    avg="N/A"
    for line in output:
        string=line.strip()
        
        if re.search('loss',string):
            loss=string[string.find('(')+1:string.find(')')-5]
        if re.search('Destination host unreachable',string):
            avg="DHU"
        elif re.search('Request timed out',string):
            avg="RTO"
        elif re.search('General failure',string):
            avg="GF"
        elif re.search('Average',string):
            avg=string[string.find('Average = ')+10:]
    temp+=f'{avg.ljust(5)} Loss={loss.ljust(4)}'
    summary.append(temp)

In [15]:
summary

['11-04-2025 07:35:14',
 'Pinged 192.168.0.1     [        Wifi        ] w/ [P128] Avg=0ms   Loss=0%  ',
 'Pinged 192.168.1.1     [        ONU         ] w/ [P128] Avg=0ms   Loss=0%  ',
 'Pinged 192.168.0.56    [      Repeater      ] w/ [P128] Avg=1ms   Loss=0%  ',
 'Pinged 172.16.109.193  [      Gateway       ] w/ [P128] Avg=1ms   Loss=0%  ',
 'Pinged 172.16.0.1      [      Old DNS       ] w/ [P128] Avg=1ms   Loss=0%  ',
 'Pinged 202.171.240.26  [      New DNS       ] w/ [P128] Avg=RTO   Loss=100%',
 'Pinged 8.8.8.8         [     Google DNS     ] w/ [P128] Avg=32ms  Loss=0%  ',
 'Pinged 1.1.1.1         [   Cloudflare DNS   ] w/ [P128] Avg=36ms  Loss=0%  ',
 'Pinged 40.126.17.132   [ Microsoft Chennai  ] w/ [P128] Avg=RTO   Loss=100%',
 'Pinged 142.250.196.46  [   Google Chennai   ] w/ [P128] Avg=42ms  Loss=0%  ',
 'Pinged 155.133.224.2   [  Valve Maharastra  ] w/ [P128] Avg=38ms  Loss=0%  ',
 'Pinged 202.78.239.186  [  Alliance Kolkata  ] w/ [P128] Avg=2ms   Loss=0%  ',
 'Pinged 57.144.